In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
from statsmodels.tsa.filters.hp_filter import hpfilter
from scipy.stats import norm
from numpy.linalg import inv, cholesky

In [243]:
from svarlr import varlr, irflr, cor, bootstrap, band

In [244]:
DATA = pd.read_excel('DATA.xlsx').rename(columns = {'Unnamed: 0' : "Time"})
data = DATA

In [245]:
country = 'US'

nchoice = 2
#(1) n: employment
#(2) n: hours

nint = 1
#(1) n=I(1)
#(0) n=I(0)

difn = 'yes'
#'yes' computed correlations based on employment growth
#'no' computed correlations based on detrended employment

LAGS=4
NVAR=2
NSTEP= 100
NDRAWS= 500
NSE = 2
IRH = 13
NW = 6


In [246]:
if nchoice==1: labor = 'LHEM' #'employment'
elif nchoice==2: labor = 'LPMHU' #'hours'
gdp = 'GDPQ'
labor_series = data[labor]

In [ ]:
print("country")
print('n: \t {labor} I({nint})')

yx = DATA[gdp]
nber = DATA.NBER #Manually entered recession start and end times according to nber
nberg = DATA.NBERG

nx = DATA[labor]

nobs = len(yx)

In [ ]:
## AR test stuff
xx = yx/nx
xx[0:4] = np.nan
xx.head(10)

# What

In [297]:
data['dlabor'] = np.log(data[labor]) - np.log(data[labor].shift(1))
data['n'] = 100 + np.log(data[labor]/data[labor][4]) * 100
data['n'][0:4] = np.nan
data['dn'] = data['dlabor']*100
data['dn'][0:5] = np.nan

data['dgdp'] = np.log(data[gdp]) - np.log(data[gdp].shift(1))
data['y'] = 100 + np.log(data[gdp]/data[gdp][4]) * 100
data['y'][0:4] = np.nan
data['dy'] = data['dgdp']*100
data['dy'][0:5] = np.nan

data['xx'] = data[gdp].div(data[labor])
data['x'] = 100 + np.log(data['xx']/data['xx'][4]) * 100
data['x'][0:4] = np.nan
data['dx'] = (np.log(data['xx']) - np.log(data['xx'].shift(1)))*100
data['dx'][0:5] = np.nan

C:\Users\joshu\AppData\Local\Temp\ipykernel_29572\2407165987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['n'][0:4] = np.nan
C:\Users\joshu\AppData\Local\Temp\ipykernel_29572\2407165987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dn'][0:5] = np.nan
C:\Users\joshu\AppData\Local\Temp\ipykernel_29572\2407165987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'][0:4] = np.nan
C:\Users\joshu\AppData\Loca

In [251]:
import statsmodels.api as sm
# Add a constant term for the intercept
ser = data.n.dropna()
X = sm.add_constant(np.arange(0,len(ser)))
model = sm.OLS(np.array(ser), X)
result = model.fit()

# Get the linear trend
trend = result.predict(X)

# Detrend the series
detrended_series = ser - trend

In [ ]:
[np.nan] * 4 + detrended_series.tolist()

In [ ]:
data.n

In [ ]:
## Correlation coefficeint
np.corrcoef(data.dx.dropna(),data.dn.dropna())[0,1]

In [ ]:
# Fit the VAR model on dy, dn, and dx
vard = data[['dy', 'dn', 'dx']].dropna()
model = VAR(vard)
var_results = model.fit(maxlags=4)  # Use maxlags similar to the RATS code, adjust as needed

# Check the summary
print(var_results.summary())

HP filter

In [202]:
# Apply HP filter to GDP and labor data
y_cycle, y_trend = hpfilter(data.y.dropna(), lamb=1600)
n_cycle, n_trend = hpfilter(data.n.dropna(), lamb=1600)
x_cycle, x_trend = hpfilter(data.x.dropna(), lamb=1600)

In [ ]:
np.corrcoef(x_cycle,n_cycle)[0,1]

In [204]:
vdata = data[['y', 'n', 'x']].dropna().reset_index(drop=True)

In [205]:
model = VAR(vdata)
var_results = model.fit(maxlags=4)

# Conditional Corr

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from numpy.linalg import cholesky


# Assuming 'data' contains the variables 'dx' and 'dn'
# 'lags' is the number of lags in the VAR model
varD = data[['dx', 'dn']].dropna()
model = VAR(varD)
var_results = model.fit(maxlags=4)  # Replace with the appropriate number of lags
residuals = var_results.resid  # Residuals are needed for shocks

# Get covariance of the residuals
sigmatR = var_results.sigma_u
sigmat = sigmatR.copy()
sigmat['dx'] = [0.43114, -0.06029]
sigmat['dn'] = [np.nan, 0.42170]

In [ ]:
sigmat

In [208]:
# Suppose 'sigmat' is the covariance matrix of the residuals from the VAR model


##### NEED TO SOLVE
emats = np.array([[0.88916, -0.49740], [0.86501, 1.73441]])

# Cholesky decomposition to get the lower triangular matrix
cmats = emats*np.array(sigmat)*emats.T

# Equivalent to RATS `smat = inv(emats)*cmats` assuming emats is the identity matrix
smat = cmats  # This is already a lower triangular matrix

# Invert smat to get sinv (similar to RATS code)
sinv = inv(smat)

In [ ]:
print(cholesky(np.array(sigmat)))
print(emats.T)
print(cmats)

In [ ]:
N = np.array([[0.58541, .29737],[-.37301, 0.53157]])
N

In [278]:
def varlrMilliken(series, p=1):
    (nobs, nvar) = series.shape
    series = np.asarray(series)
    YY = series[p:nobs]
    XX = np.ones([nobs - p, 1 + nvar * p])
    
    for i in range(1, p + 1):
        start_col = 1 + nvar * (i - 1)
        end_col = 1 + nvar * i
        XX[:, start_col:end_col] = series[p - i:nobs - i, :]
    
    # Estimate VAR coefficients
    b = np.linalg.lstsq(XX, YY, rcond=None)[0]
    e = YY - np.dot(XX, b)
    Sig = np.dot(e.conj().transpose(), e) / (nobs - p)
    
    # Extract AR Coefficient Matrices
    AR_matrices = []
    for i in range(p):
        start_idx = 1 + i * nvar
        end_idx = 1 + (i + 1) * nvar
        Ai = b[start_idx:end_idx, :].conj().transpose()
        AR_matrices.append(Ai)
    
    # Sum of AR Coefficients
    A_sum = np.zeros((nvar, nvar))
    for Ai in AR_matrices:
        A_sum += Ai
    
    # Compute Phi1
    Phi1 = np.linalg.inv(np.identity(nvar) - A_sum)
    
    # Compute mu and lvar
    mu = np.dot(Phi1, b[0].conj().transpose())
    lvar = np.dot(Phi1, np.dot(Sig, Phi1.conj().transpose()))
    
    # Long-Run restriction
    theta1 = np.linalg.cholesky(lvar.conj().transpose())
    B = np.dot(np.linalg.inv(Phi1), theta1)
    
    return (b, Sig, B, e)

In [ ]:
from statsmodels.tsa.vector_ar.svar_model import SVAR
# Now, we set up the matrices needed for identification
A = np.array([['E', 0],
              ['E', 'E']])

# We can estimate the SVAR with the long-run restriction using statsmodels
svar_model = SVAR(varD, svar_type='A', A=A, )

In [ ]:
# Prepare the long-run restriction matrix
A_longrun = np.array([['E', 0],
                      ['E', 'E']])

# Estimate the SVAR model with long-run restrictions
svar_model = SVAR(varD, svar_type='A', A=A_longrun)
svar_results = svar_model.fit(maxlags = LAGS)

# Step 8: Impulse Responses
# Compute impulse responses up to NSTEP periods
irf = svar_results.irf(NSTEP)
irfs = irf.irfs  # Shape: (NSTEP+1, nvars, nvars)

# Extract the impulse responses
c11 = irfs[:, 0, 0]  # Response of 'dx' to technology shock
c12 = irfs[:, 0, 1]  # Response of 'dx' to non-technology shock
c21 = irfs[:, 1, 0]  # Response of 'dnz' to technology shock
c22 = irfs[:, 1, 1]  # Response of 'dnz' to non-technology shock

# Adjust 'c21' and 'c22' if 'nint' == 0 and 'difn' == 'yes'
if nint == 0 and difn == 'yes':
    c21_adjusted = np.concatenate(([c21[0]], np.diff(c21)))
    c22_adjusted = np.concatenate(([c22[0]], np.diff(c22)))
    c21 = c21_adjusted
    c22 = c22_adjusted

# Compute cumulative impulse responses
c11s = np.cumsum(c11)
c12s = np.cumsum(c12)
c21s = np.cumsum(c21)
c22s = np.cumsum(c22)

# Step 9: Historical Decomposition
# Compute the structural shocks
eps = svar_results.resid  # Structural shocks: [eps1, eps2]

In [ ]:
e.shape

In [ ]:
eps

In [ ]:
b, Sig, B, e = varlrMilliken(series = varD, p=4)
print(b)
print(f"Sig {Sig}")
print(B)
print(e)


In [281]:
irfs = irflr(b,B,IRH)

In [282]:

# Step 9: Compute conditional variances and covariances using MA coefficients
# For productivity ('dx') and hours ('dnz')

# Extract MA coefficients for 'dx' and 'dnz' responses to each shock
# Let's denote:
# - C_11: response of 'dx' to technology shock
# - C_12: response of 'dx' to non-technology shock
# - C_21: response of 'dnz' to technology shock
# - C_22: response of 'dnz' to non-technology shock

C_11 = irfs[0]  # 'dx' response to tech shock
C_12 = irfs[2]  # 'dx' response to non-tech shock
C_21 = irfs[1]  # 'dnz' response to tech shock
C_22 = irfs[3]  # 'dnz' response to non-tech shock


In [ ]:
np.diff(C_21)

In [ ]:
Cs = pd.read_excel("Cs.xlsx")
"""


C_11 = Cs.c11  # 'dx' response to tech shock
C_12 = Cs.c12  # 'dx' response to non-tech shock
C_21 = Cs.c21  # 'dnz' response to tech shock
C_22 = Cs.c22 
"""

In [302]:
def simulate_var_contribution(AR_matrices, B, eps_shock, p, initial_values):
    T = eps_shock.shape[0]
    nvar = B.shape[0]
    Y_sim = np.zeros((T, nvar))
    Y_sim[:p, :] = initial_values
    for t in range(p, T):
        Y_t = np.zeros(nvar)
        for i in range(p):
            Y_t += AR_matrices[i] @ Y_sim[t - i - 1, :]
        Y_t += B @ eps_shock[t, :]
        Y_sim[t, :] = Y_t
    return Y_sim

In [303]:
detrend = 0
corr_uncon = np.corrcoef(data.dx.dropna(), data.dn.dropna())[0,1]
varD = data[['dx', 'dn']].dropna()
b, Sig, B, e = varlrMilliken(series=varD, p=4)
irfs = irflr(b, B, IRH)
# Existing code to compute impulse responses...

# Compute structural shocks
eps = np.linalg.solve(B, e.T).T  # Shape: (T, nvar)

# Extract VAR coefficient matrices
p = 4  # Number of lags
nvar = 2  # Number of variables
AR_matrices = []
for i in range(p):
    start_idx = 1 + i * nvar
    end_idx = 1 + (i + 1) * nvar
    Ai = b[start_idx:end_idx, :].T
    AR_matrices.append(Ai)

# Initial values
initial_values = varD.values[:p, :]

# Simulate contributions from each shock
contributions = []
for j in range(nvar):
    eps_shock = np.zeros_like(eps)
    eps_shock[:, j] = eps[:, j]
    Y_sim = simulate_var_contribution(AR_matrices, B, eps_shock, p, initial_values)
    contributions.append(Y_sim)
contributions = np.stack(contributions, axis=2)  # (T, nvar, nvar)

# Extract contributions
hd11 = contributions[:, 0, 0]
hd12 = contributions[:, 1, 0]
hd21z = contributions[:, 0, 1]
hd22z = contributions[:, 1, 1]

# Adjust contributions based on detrending
if detrend == 1:
    hd21 = np.zeros_like(hd21z)
    hd22 = np.zeros_like(hd22z)
    hd21[0] = hd21z[0]
    hd22[0] = hd22z[0]
    hd21[1:] = hd21z[1:] - hd21z[:-1]
    hd22[1:] = hd22z[1:] - hd22z[:-1]
else:
    hd21 = hd21z
    hd22 = hd22z

# Compute total contributions
hd31 = hd21 + hd11
hd32 = hd22 + hd12

# Assign contributions to variables
dyt = hd31
dnt = hd21
dxt = hd11
dyd = hd32
dnd = hd22
dxd = hd12

# Compute cumulative sums
yt = np.cumsum(dyt)
yd = np.cumsum(dyd)
xt = np.cumsum(dxt)
xd = np.cumsum(dxd)
nt = np.cumsum(dnt)
nd = np.cumsum(dnd)

# Adjust nt and nd if needed
if detrend == 1:
    nt = dnt
    nd = dnd

In [306]:
dnt

array([ 1.07160785e+00, -3.91779933e-01,  1.49111893e+00,  1.01564341e+00,
        9.58147109e-01,  9.85452753e-01, -2.45031098e-01,  4.48522037e-02,
       -9.04903747e-04,  3.65364630e-02, -6.92160863e-01, -2.50050474e-01,
       -4.85105048e-01,  4.25448657e-01,  6.17305351e-01,  4.06366021e-02,
       -2.47779498e-01, -5.33461484e-01, -1.87557306e-01, -1.73875626e-01,
        3.82859289e-01,  2.56488726e-01,  4.99477721e-01,  1.86080662e-01,
       -1.20260754e-01, -3.70048411e-01,  1.52862762e-01, -4.75025057e-01,
        2.17162715e-01, -5.13967438e-02,  5.46121313e-01, -4.98408489e-01,
       -1.31779119e-01,  6.10550505e-02, -6.54143257e-01, -4.88197197e-01,
        7.16470214e-01,  1.02774629e+00,  8.83309301e-02, -9.08994291e-02,
       -7.06467792e-02, -7.99971095e-01,  3.95620375e-01,  3.58022208e-01,
       -5.17159755e-01, -1.45499925e-01, -2.88696874e-01,  4.36644625e-01,
        3.75322904e-01,  8.86521966e-02,  9.23580112e-02, -2.35585228e-01,
        7.36885447e-03, -

In [218]:
C_11s = np.cumsum(C_11)
C_12s = np.cumsum(C_12)
C_21s = np.cumsum(C_21)
C_22s = np.cumsum(C_22)

In [ ]:
# Compute variances of 'dx' and 'dnz' for each component
# Variance due to technology shock
var_dx_tech = np.sum(C_11 ** 2)
var_dnz_tech = np.sum(C_21 ** 2)
cov_dx_dnz_tech = np.sum(C_11 * C_21)

# Variance due to non-technology shock
var_dx_nontech = np.sum(C_12 ** 2)
var_dnz_nontech = np.sum(C_22 ** 2)
cov_dx_dnz_nontech = np.sum(C_12 * C_22)

# Compute conditional correlations
corr_tech = cov_dx_dnz_tech / np.sqrt(var_dx_tech * var_dnz_tech)
corr_nontech = cov_dx_dnz_nontech / np.sqrt(var_dx_nontech * var_dnz_nontech)

print("Conditional Correlation (Technology Shock):", corr_tech)
print("Conditional Correlation (Non-Technology Shock):", corr_nontech)

# Step 10: Generate Figure 1 - Scatter plots of innovations
# Obtain structural shocks (innovations)
#epsilon = svar_results.resid

# Note: Since the shocks are already estimated, we can use them directly



In [ ]:
np.array([C_11,C_21])

# -----------------------------

In [ ]:
ax = sns.scatterplot(
    data=data[['dn','dx']].dropna(), 
    x='dn', 
    y='dx', 
    marker='s',        # 's' is the marker style for squares
    color='black',     # Set color to black
    edgecolor='black'  # Set edge color to black (useful if the color of the marker is different)
)



plt.xlabel('Hours', fontsize=14)
plt.ylabel('Productivity', fontsize=14)
plt.title('Data', fontsize=16)

In [ ]:

# Plot impulse responses to technology shock
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 12))
axes[0].plot(C_11s[:IRH], label='dx response to Technology Shock')
axes[0].set_title('Impulse Response of dx to Technology Shock')
axes[0].set_xlabel('Periods')
axes[0].set_ylabel('Response')
axes[0].legend()

axes[1].plot(C_21s[:IRH], label='dnz response to Technology Shock')
axes[1].set_title('Impulse Response of dnz to Technology Shock')
axes[1].set_xlabel('Periods')
axes[1].set_ylabel('Response')
axes[1].legend()

axes[2].plot((C_11s + C_21s)[:IRH], label='(dx + dnz) response to Technology Shock')
axes[2].set_title('Impulse Response of (dx + dnz) to Technology Shock')
axes[2].set_xlabel('Periods')
axes[2].set_ylabel('Response')
axes[2].legend()

plt.tight_layout()
plt.show()

# Plot impulse responses to non-technology shock
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 12))
axes[0].plot(C_12s[:IRH], label='dx response to Non-Technology Shock')
axes[0].set_title('Impulse Response of dx to Non-Technology Shock')
axes[0].set_xlabel('Periods')
axes[0].set_ylabel('Response')
axes[0].legend()

axes[1].plot(C_22s[:IRH], label='dnz response to Non-Technology Shock')
axes[1].set_title('Impulse Response of dnz to Non-Technology Shock')
axes[1].set_xlabel('Periods')
axes[1].set_ylabel('Response')
axes[1].legend()

axes[2].plot((C_12s + C_22s)[:IRH], label='(dx + dnz) response to Non-Technology Shock')
axes[2].set_title('Impulse Response of (dx + dnz) to Non-Technology Shock')
axes[2].set_xlabel('Periods')
axes[2].set_ylabel('Response')
axes[2].legend()

plt.tight_layout()
plt.show()